# Step 2: Preprocessing & Classification model
This section will load up the defined settings from the pickles directory and run the machine learning pipeline with the help of the `pycaret` library and save respective data.

In [1]:
# importing all packages needed in this section
import pandas as pd
import os
import sys 

from sklearn.metrics import classification_report
from pycaret.classification import *

# utility functions for the experiment
sys.path.append('../src')

from mlflow_manager import MLFlowManager
from utils import getPicklesFromDir, getExperimentConfig, run_pycaret_setup, translate_model_name

# Get global experiment settings
config = getExperimentConfig()
folders = config['folders']
# get a list of all settings for the datasets prepared beforehand
dataset_settings = getPicklesFromDir(folders['settings_dir'])  

In [2]:
import time

start_time = time.perf_counter()

dataset_settings pickle is saved as follows:
```
"meta_data": meta_dataset,  # contains information about the dataset, including path
"setup_param": setup_param, # contains all the setup parameters for pycaret setup() function
"sdg_param": sdg_param,     # contains all sdg parameters for the CTGAN() function

```

In [3]:
for settings in dataset_settings:
    # get path
    dataset_path = f"{folders['real_dir']}{settings['meta']['filename']}"
    # run setup function
    s = run_pycaret_setup(dataset_path, settings['setup_param'])
    
    USI = s.get_config('USI')

    
    # Init experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"
    mlflow = MLFlowManager(experiment_name)
    
    logg_tags = {
        'USI': USI,
        'Dataset ID': settings['meta']['id'],
        'Dataset Type': 'original'
    }
    
    mlflow.start_run("Original data models", tags=logg_tags)
    
    # for each defined model in the global config
    # create specified model and tune it
    for ml_model in config['clf']['ml_models']:
        
        model_name = f"{settings['meta']['id']}-{translate_model_name(ml_model)}"
        logg_tags['model']=ml_model
        
        mlflow.start_run(model_name, tags=logg_tags, nested=True)

        # create & tune model
        model = s.create_model(ml_model)
        tuned_model = s.tune_model( model, **config['clf']['tuning_param'] )
        
        # get validation results
        val_df = s.pull()
        val_score = {}
        val_score['Accuracy'] = val_df['Accuracy']['Mean']
        val_score['F1-score'] = val_df['F1']['Mean']
        val_score['AUC']      = val_df['AUC']['Mean']
        val_score['Kappa']    = val_df['Kappa']['Mean']
        val_score['MCC']      = val_df['MCC']['Mean']
        
        # test the model on the holdout-data
        holdout_score = s.predict_model(estimator=tuned_model)
        #metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
        #metrics_df = pd.DataFrame(metrics).transpose()
        
        # log parameters     
        mlflow.log_params(tuned_model.get_params())
        # log performance
        mlflow.log_metrics(val_score)
        mlflow.log_score_report_to_html(val_df, "Validation")
        mlflow.log_score_report_to_html(holdout_score, "Holdout")
        # log model
        mlflow.log_model(model=tuned_model)
        
        mlflow.end_run()
        
    # Save model details on the model with best accurracy under the the 'Original data models' run
    best_run = mlflow.get_best_run_by_metric(metric_name='Accuracy')
    
    mlflow.log_params(best_run.data.params)
    mlflow.log_metrics(best_run.data.metrics)
    mlflow.log_tag('model run name', best_run.data.tags['mlflow.runName'])
    mlflow.log_tag('model', best_run.data.tags['model'])
    mlflow.log_tag('model run id', best_run.info.run_id)
    
    mlflow.end_run()          

,Description,Value
0,Session id,6544
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(614, 9)"
6,Transformed test set shape,"(154, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8065,0.8818,0.6364,0.7778,0.7000,0.5592,0.5654
1,0.7258,0.7807,0.5455,0.6316,0.5854,0.3822,0.3845
2,0.8226,0.8875,0.6364,0.8235,0.7179,0.5916,0.6021
3,0.7097,0.7773,0.4545,0.6250,0.5263,0.3245,0.3330
4,0.7213,0.7762,0.5238,0.6111,0.5641,0.3611,0.3634
5,0.8033,0.8726,0.6667,0.7368,0.7000,0.5542,0.5557
6,0.7869,0.8417,0.4762,0.8333,0.6061,0.4745,0.5094
7,0.7705,0.7429,0.5238,0.7333,0.6111,0.4547,0.4676
8,0.7705,0.8762,0.5714,0.7059,0.6316,0.4676,0.4731


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8065,0.8818,0.6364,0.7778,0.7000,0.5592,0.5654
1,0.7258,0.7830,0.5455,0.6316,0.5854,0.3822,0.3845
2,0.8226,0.8875,0.6364,0.8235,0.7179,0.5916,0.6021
3,0.7097,0.7750,0.4545,0.6250,0.5263,0.3245,0.3330
4,0.7213,0.7798,0.5238,0.6111,0.5641,0.3611,0.3634
5,0.8033,0.8726,0.6667,0.7368,0.7000,0.5542,0.5557
6,0.7869,0.8393,0.4762,0.8333,0.6061,0.4745,0.5094
7,0.7705,0.7429,0.5238,0.7333,0.6111,0.4547,0.4676
8,0.7869,0.8762,0.6190,0.7222,0.6667,0.5114,0.5147


Fitting 10 folds for each of 2 candidates, totalling 20 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7258,0.7869,0.5455,0.6316,0.5854,0.3822,0.3845
1,0.6774,0.7614,0.5000,0.5500,0.5238,0.2807,0.2815
2,0.7581,0.8455,0.5000,0.7333,0.5946,0.4308,0.4469
3,0.7097,0.7812,0.5455,0.6000,0.5714,0.3527,0.3536
4,0.6557,0.6798,0.4286,0.5000,0.4615,0.2107,0.2121
5,0.7541,0.8155,0.4762,0.7143,0.5714,0.4085,0.4250
6,0.6721,0.7512,0.2857,0.5455,0.3750,0.1812,0.1986
7,0.7705,0.7518,0.6667,0.6667,0.6667,0.4917,0.4917
8,0.7705,0.8036,0.6667,0.6667,0.6667,0.4917,0.4917


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7581,0.8477,0.7727,0.6296,0.6939,0.4973,0.5044
1,0.8065,0.8091,0.6364,0.7778,0.7000,0.5592,0.5654
2,0.8226,0.8909,0.6364,0.8235,0.7179,0.5916,0.6021
3,0.7258,0.8648,0.5455,0.6316,0.5854,0.3822,0.3845
4,0.7541,0.7202,0.6190,0.6500,0.6341,0.4491,0.4494
5,0.7705,0.8548,0.6190,0.6842,0.6500,0.4799,0.4812
6,0.8361,0.8667,0.5714,0.9231,0.7059,0.6008,0.6340
7,0.6721,0.7464,0.4762,0.5263,0.5000,0.2570,0.2577
8,0.7869,0.8702,0.5238,0.7857,0.6286,0.4874,0.5071


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7742,0.8500,0.7273,0.6667,0.6957,0.5167,0.5179
1,0.7903,0.8193,0.6818,0.7143,0.6977,0.5373,0.5377
2,0.7903,0.8648,0.5455,0.8000,0.6486,0.5067,0.5256
3,0.7258,0.7875,0.6364,0.6087,0.6222,0.4072,0.4074
4,0.7049,0.7583,0.6190,0.5652,0.5909,0.3609,0.3618
5,0.8361,0.8679,0.7143,0.7895,0.7500,0.6285,0.6302
6,0.8361,0.8488,0.5714,0.9231,0.7059,0.6008,0.6340
7,0.6885,0.7393,0.4762,0.5556,0.5128,0.2859,0.2877
8,0.7705,0.8452,0.5238,0.7333,0.6111,0.4547,0.4676


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6452,0.8568,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.6452,0.8227,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.6452,0.8455,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.6452,0.8426,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.6557,0.6815,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.6557,0.8077,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.6557,0.8000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.6557,0.7452,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.6557,0.8827,0.0000,0.0000,0.0000,0.0000,0.0000


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [11]:
s.get_leaderboard()

Pipeline(memory=FastMemory(location=C:\Users\flore\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['DiabetesPedigreeFunction', 'BMI',
                                             'Insulin', 'Glucose', 'Age',
                                             'SkinThickness', 'BloodPressure',
                                             'Pregnancies'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              missing_values=nan,
                                                              strate...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                                                               with_std=True))),
                ['trained_model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=6544,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False)]],
         verbose=False)

---

### Notice 
Following cells until end of section (i.e. section 3.0) contains experimental code that will not be run.

In [16]:
### Following shows which are models are natively available in the pycaret library
# It is possible to add estimators
all_models = models()
display(all_models)

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessClassifier,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [17]:
##### How to get logs, can pass experiment name to get_logs()
#get_logs()[[
#    'tags.mlflow.runName',
#    'metrics.Accuracy',
#    'metrics.F1',
#    'metrics.Prec',
#    'metrics.Recall',
#    'tags.Source']].sort_values('metrics.Accuracy', ascending=False)

In [18]:
# Need to define param that should be explored, define which method e.g. grid_search vs random vs optuna
# default search method: random grid search
# Todo: lookup default search range parameters

# uses the best model to optimze
#tuned = tune_model(clf, optimize='Accuracy', n_iter=10)

---

# Step 3: SDG

In [ ]:
# necessary imports for the section
from sdv.tabular import CTGAN

import pickle
import pandas as pd 
import os 
import sys

sys.path.append('../src')
from utils import (getPicklesFromDir, 
                   getExperimentConfig, 
                   extract_loss_info_from_stdout, 
                   create_loss_plot)

from mlflow_manager import MLFlowManager

# Get global experiment settings
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Get dataset specific settings
dataset_settings = getPicklesFromDir(folders['settings_dir'])

Psuedo code:

```
for each pickle (setting):
    for each varying quality:
        create model with sdg_param and quality
        train model with original_data
        generate num_SD synthetic datasets with:
            num_rows=SD_size_by_factor * len(original_dataset)
        
        save synthetic dataset in dataset folder
        save SDG
```

In [ ]:
%%capture stdout_loss

# get settings
quality_params = config['ctgan_param']['quality_params']
sd_size_factor = config['ctgan_param']['sd_size_factor']
num_SD = config['ctgan_param']['num_sd']


# run SDG generation
# for each dataset specific settings
for s_index, settings in enumerate(dataset_settings):
    # Init experiment logging
    experiment_name = f"{settings['meta']['id']}-SDG-models"
    mlflow = MLFlowManager(experiment_name)
    
    # load original dataset
    original_data = pd.read_csv(folders['real_dir'] + settings['meta']['filename'])  
    
    # get the size to generate the synthetic data
    original_data_size = len(original_data)
    sd_size = original_data_size * sd_size_factor
    
    logg_tags = {'Source': settings['meta']['id']}
    
    # loop through the different quality parameters for the SDG
    for quality in quality_params:
        
        sdg_name = f"S{settings['meta']['id']}{quality}"
        log_run = mlflow.start_run(sdg_name, tags=logg_tags)
        mlflow.log_params(quality_params[quality])
        
        # creates model with sdg_param and quality_param as parameters
        print("#START#")   # for capturing loss info
        
        model = CTGAN(**settings['sdg_param'], **quality_params[quality])
        
        print(sdg_name)   # for capturing loss info
        model.fit(original_data)
        
        # saves the SDG model using cloudpickle
        model_file = f"{sdg_name}.pkl"
        model.save(model_file) 
        mlflow.log_artifact(model_file)
        
        if os.path.isfile(model_file):
            os.remove(model_file)
        
        print("#END#")   # for capturing loss info
        mlflow.end_run()
        
        # create num_SD SDGs and synthetic datasets for validating results
        for itr in range(num_SD):
            
            # creates Synthetic dataset name, using datset id, quality key, and itr number 
            # e.g. SD1Q1_2 means SDG trained on datset D1 with quality Q1 and copy num 2
            SD_name = f"S{settings['meta']['id']}{quality}_{str(itr)}"
            
            # relative file path for the synthetic dataset
            sd_path = f"{folders['sd_dir']}{SD_name}.csv"
            
            # generate synthetic data
            synthetic_data = model.sample(num_rows=sd_size)
            
            # save the synthetic dataset
            synthetic_data.to_csv(sd_path, index=False)
            
    mlflow.end_run()

In [ ]:
""" The Loss values captured from the cell above's standard output will 
be used to create the generator vs discriminator loss plots.
"""
loss_values = extract_loss_info_from_stdout(stdout_loss.stdout)

for sdg_id in loss_values:
    fig = create_loss_plot(sdg_id, loss_values[sdg_id])
    # Save the plot to correct mlflow log
    run=mlflow.load_run_by_name(sdg_id)
    path=run.info.artifact_uri.replace("file:///", "")
    #save plot
    fig.savefig(f"{path}/{sdg_id}_loss_plot.png")
    #save data
    loss_values.to_csv(f"{path}/{sdg_id}.csv", index=False)


---

# Step 4: Create models with SD

In [ ]:
import pandas as pd 
import os 
import sys 
import pickle


from sklearn.metrics import (classification_report, 
                             roc_auc_score, 
                             matthews_corrcoef,
                             cohen_kappa_score)
sys.path.append('../src')
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   run_pycaret_setup, 
                   translate_model_name,
                   get_synthetic_filepaths_from_original_data_id,
                   convert_and_clean_dict)

from mlflow_manager import MLFlowManager

# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

In [23]:
# TODO: consider testing, and consider extracting hyperparameters, consider how to summarize
for settings in dataset_settings:
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False
    
    # Get experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"
    mlflow = MLFlowManager(experiment_name)
    
    best_model_from_original_data = mlflow.get_best_model_hyperparameters()

    original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}")
    y_test = original_data[settings['meta']['target']]
    x_test = original_data.drop(columns=[settings['meta']['target']])

    logg_tags = {'Dataset Type': 'synthetic'}
    mlflow.start_run('Synthetic data models', tags=logg_tags)
    
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

    for sd_filename in synthetic_datasets:
        
        sd_id = os.path.splitext(sd_filename)[0]
        sd_path = folders['sd_dir']+sd_filename
        
        s = run_pycaret_setup(sd_path, settings['setup_param'])
        USI = s.get_config('USI')
        
        # Add custom tags to the logg, defining dataset type, and Id
        logg_tags = {
            'Dataset Type': 'synthetic',
            'Dataset ID': sd_id,
            'USI': USI
        }
        run_name = f"{sd_id}-models"
        mlflow.start_run(run_name, tags=logg_tags, nested=True)

        # Test the model with best performance from best original dataset
        ml_model = best_model_from_original_data['model']
        hyperparameters = best_model_from_original_data['params']
        hyperparameters = convert_and_clean_dict(hyperparameters)
        model_name = f"{sd_id}-Orignal_{translate_model_name(ml_model)}"
        logg_tags['model']=model_name
        
        mlflow.start_run(model_name, tags=logg_tags, nested=True)

        # create & tune model
        tmp_model = s.create_model(ml_model)
        model=tmp_model.set_params(**hyperparameters)
        tuned_model = s.create_model(model)
    
        # get validation results
        val_df = s.pull()
        val_score = {}
        val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
        val_score['val_F1-score'] = val_df['F1']['Mean']


        pred_model = s.predict_model(estimator=tuned_model, data=x_test)
        y_pred = pred_model['prediction_label']

        metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
        test_score = pd.DataFrame.from_dict(metrics).transpose()
        
        test_metrics = {
            "Accuracy": metrics['accuracy'],
            "F1-score": metrics['macro avg']['f1-score'],
            "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
            "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
        }

        # If there is a prediction_score in the from predict_model (sometimes there isn't)
        if 'prediction_score' in pred_model.columns:
            y_pred_score = pred_model['prediction_score']
            # If multiclass classification, set argument multi_class='one-vs-one'
            if original_data[settings['meta']['target']].nunique() > 2:
                m_class = 'ovo'
            else:
                m_class = 'raise'
            test_metrics['AUC'] = roc_auc_score(y_true=y_test, y_score=y_pred_score, multi_class=m_class)


        # log parameters     
        mlflow.log_params(tuned_model.get_params())
        # log performance
        mlflow.log_metric_report(test_metrics)
        mlflow.log_metrics(val_score)
        mlflow.log_score_report_to_html(val_df, "Validation")
        mlflow.log_score_report_to_html(test_score, "Test_score")
        # log model
        mlflow.log_model(model=tuned_model)
        # end logging run for the model
        mlflow.end_run()
        
        # Start testing all models
        for ml_model in config['clf']['ml_models']:
            #start log run
            logg_tags['model']=ml_model
            model_name = f"{sd_id}-{translate_model_name(ml_model)}"
            mlflow.start_run(model_name, tags=logg_tags, nested=True)
            
            # create & tune model
            model = s.create_model(ml_model)
            tuned_model = s.tune_model(model, **config['clf']['tuning_param']) 
        
            # get validation results
            val_df = s.pull()
            val_score = {}
            val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
            val_score['val_F1-score'] = val_df['F1']['Mean']


            pred_model = s.predict_model(estimator=tuned_model, data=x_test)
            y_pred = pred_model['prediction_label']

            metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
            test_score = pd.DataFrame.from_dict(metrics).transpose()
            
            test_metrics = {
                "Accuracy": metrics['accuracy'],
                "F1-score": metrics['macro avg']['f1-score'],
                "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
            }

            # If there is a prediction_score in the from predict_model (sometimes there isn't)
            if 'prediction_score' in pred_model.columns:
                y_pred_score = pred_model['prediction_score']
                # If multiclass classification, set argument multi_class='one-vs-one'
                if original_data[settings['meta']['target']].nunique() > 2:
                    m_class = 'ovo'
                else:
                    m_class = 'raise'
                test_metrics['AUC'] = roc_auc_score(y_true=y_test, y_score=y_pred_score, multi_class=m_class)


            # log parameters     
            mlflow.log_params(tuned_model.get_params())
            # log performance
            mlflow.log_metric_report(test_metrics)
            mlflow.log_metrics(val_score)
            mlflow.log_score_report_to_html(val_df, "Validation")
            mlflow.log_score_report_to_html(test_score, "Test_score")
            # log model
            mlflow.log_model(model=tuned_model)
            # end logging run for the model
            mlflow.end_run()
            
        # end logging run for SD_id
        #TODO: save best model for each sd_id-models
        # Save model details on the model with best accurracy under the the 'Original data models' run
        #best_run = mlflow.get_best_run_by_metric(metric_name='Accuracy')

        #mlflow.log_params(best_run.data.params)
        #mlflow.log_metrics(best_run.data.metrics)
        #mlflow.log_tag('model run name', best_run.data.tags['mlflow.runName'])
        #mlflow.log_tag('model', best_run.data.tags['model'])
        #mlflow.log_tag('model run id', best_run.info.run_id)
    
        mlflow.end_run()

    # end logging for the synthetic datasets based on original id
    mlflow.end_run()          

,Description,Value
0,Session id,559
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(922, 9)"
4,Transformed data shape,"(1230, 9)"
5,Transformed train set shape,"(922, 9)"
6,Transformed test set shape,"(308, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6452,0.6994,0.5435,0.6757,0.6024,0.2888,0.2944
1,0.6452,0.6574,0.5556,0.6579,0.6024,0.2861,0.2895
2,0.6413,0.6624,0.5111,0.6765,0.5823,0.2785,0.2869
3,0.5761,0.6274,0.5111,0.5750,0.5412,0.1498,0.1507
4,0.6957,0.7745,0.7333,0.6735,0.7021,0.3922,0.3937
5,0.6957,0.7447,0.7111,0.6809,0.6957,0.3916,0.3920
6,0.5870,0.6411,0.6667,0.5660,0.6122,0.1766,0.1794
7,0.6304,0.6842,0.6000,0.6279,0.6136,0.2598,0.2601
8,0.7065,0.8156,0.7333,0.6875,0.7097,0.4136,0.4145


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6452,0.6994,0.5435,0.6757,0.6024,0.2888,0.2944
1,0.6559,0.6588,0.5556,0.6757,0.6098,0.3073,0.3120
2,0.6413,0.6629,0.5111,0.6765,0.5823,0.2785,0.2869
3,0.5761,0.6274,0.5111,0.5750,0.5412,0.1498,0.1507
4,0.6957,0.7740,0.7333,0.6735,0.7021,0.3922,0.3937
5,0.6957,0.7452,0.7111,0.6809,0.6957,0.3916,0.3920
6,0.5870,0.6411,0.6667,0.5660,0.6122,0.1766,0.1794
7,0.6304,0.6837,0.6000,0.6279,0.6136,0.2598,0.2601
8,0.7065,0.8156,0.7333,0.6875,0.7097,0.4136,0.4145


Fitting 10 folds for each of 2 candidates, totalling 20 fits


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.6526,0.6882,0.5693,0.6190,0.5932,0.2910,0.2917


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7957,0.9070,0.7826,0.8000,0.7912,0.5913,0.5914
1,0.8172,0.8848,0.7826,0.8372,0.8090,0.6341,0.6354
2,0.8261,0.8832,0.7778,0.8537,0.8140,0.6514,0.6538
3,0.8261,0.8671,0.8667,0.7959,0.8298,0.6527,0.6551
4,0.8478,0.9395,0.8478,0.8478,0.8478,0.6957,0.6957
5,0.8804,0.9466,0.8478,0.9070,0.8764,0.7609,0.7625
6,0.9239,0.9792,0.9783,0.8824,0.9278,0.8478,0.8529
7,0.8696,0.9627,0.8478,0.8864,0.8667,0.7391,0.7398
8,0.8043,0.9159,0.8043,0.8043,0.8043,0.6087,0.6087


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8387,0.9070,0.8261,0.8444,0.8352,0.6773,0.6775
1,0.7742,0.8696,0.7174,0.8049,0.7586,0.5478,0.5510
2,0.8043,0.8816,0.7778,0.8140,0.7955,0.6081,0.6087
3,0.8043,0.8837,0.8889,0.7547,0.8163,0.6100,0.6194
4,0.8913,0.9405,0.9565,0.8462,0.8980,0.7826,0.7894
5,0.8804,0.9499,0.8913,0.8723,0.8817,0.7609,0.7610
6,0.8696,0.9660,0.8913,0.8542,0.8723,0.7391,0.7398
7,0.8804,0.9442,0.8478,0.9070,0.8764,0.7609,0.7625
8,0.8370,0.9230,0.8261,0.8444,0.8352,0.6739,0.6741


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.9156,0.9656,0.9179,0.8913,0.9044,0.8289,0.8291


---

# Step 5: Measure Population Fidelity (PF)

In [24]:
import pandas as pd 
import sys
import os

sys.path.append('../src')
from PF_metrics import compute_all_pf_measures
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   get_synthetic_filepaths_from_original_data_id)

config = getExperimentConfig()
folders = config['folders']

settings = getPicklesFromDir(folders['settings_dir'])

In [25]:
pf_measure_columns = [
    'DatasetName',
    'pMSE',
    'pMSE_time',
    's_pMSE',
    's_pMSE_time',
    'Cluster_1',   # num of clusters = 1% of dataset_size
    'Cluster_1_time',   # num of clusters = 1% of dataset_size
    #'Cluster_5',     # num of clusters = 5% of dataset_size
    #'Cluster_10',    # num of clusters = 10% of dataset_size
    'BNLogLikelihood',
    'BNLogLikelihood_time',
    'GMLogLikelihood',
    'GMLogLikelihood_time',
    'ContinousKLDivergence',
    'ContinousKLDivergence_time',
    'DiscreteKLDivergence',
    'DiscreteKLDivergence_time',
    'KSComplement',
    'KSComplement_time',
    'CSTest',
    'CSTest_time',
    'CrossClassification', #Cross-classification 
    'CrossClassification_time', #Cross-classification 
]


result_df = pd.DataFrame(columns=pf_measure_columns)

for dataset_setting in settings:
    original_data = pd.read_csv(folders['real_dir']+dataset_setting['meta']['filename'])
    
    metadata = dataset_setting['meta']['meta_data']
    original_data_id = dataset_setting['meta']['id']
    
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(original_data_id)
    
    
    for sd_filename in synthetic_datasets:
        
        sd_id = os.path.splitext(sd_filename)[0]
        sd_path = folders['sd_dir']+sd_filename

        synthetic_data = pd.read_csv(sd_path)
        
        pf_measures = compute_all_pf_measures(original_data=original_data,
                                              synthetic_data=synthetic_data,
                                              metadata=metadata,
                                              SD_id=sd_id)
        
        result_df = pd.concat([result_df, pf_measures], axis=0, ignore_index=True)


# save the results
result_df.to_csv('../data/pf_measures.csv', index=False)
result_df.to_html('../data/pf_measures.html')

num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>
num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>
num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>
num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>
num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>
num samples data: 1536, num_klusters:15
(1536, 9)
<class 'pandas.core.frame.DataFrame'>


In [26]:
display(result_df)

,DatasetName,pMSE,pMSE_time,s_pMSE,s_pMSE_time,Cluster_1,Cluster_1_time,BNLogLikelihood,BNLogLikelihood_time,GMLogLikelihood,...,ContinousKLDivergence,ContinousKLDivergence_time,DiscreteKLDivergence,DiscreteKLDivergence_time,KSComplement,KSComplement_time,CSTest,CSTest_time,CrossClassification,CrossClassification_time
0,SD0Q1_0,0.230725,0.021544,223.564075,0.025691,22.642776,11.974487,-0.751548,0.144707,-28.667452,...,0.377548,0.082617,NaN,0.000251,0.534993,0.005441,0.724537,0.000837,0.433939,0.094462
1,SD0Q1_1,0.225934,0.018198,210.366665,0.018104,21.753316,13.063949,-0.730436,0.030630,-32.838181,...,0.321741,0.083083,NaN,0.001183,0.522461,0.009147,0.778423,0.001320,0.438625,0.120435
2,SD0Q1_2,0.231650,0.018921,219.655623,0.021202,21.883990,11.873773,-0.712572,0.034391,-32.815682,...,0.330569,0.080604,NaN,0.000289,0.515625,0.007362,0.824878,0.001012,0.464286,0.101539
3,SD0Q2_0,0.030482,0.020946,26.331309,0.025975,1.566102,15.072511,-0.766164,0.039285,-27.012169,...,0.652129,0.079691,NaN,0.000292,0.856934,0.007170,0.687998,0.001047,0.647746,0.128270
4,SD0Q3_0,0.054103,0.026947,45.735094,0.022852,4.883650,18.093950,-0.736932,0.030407,-32.410436,...,0.461794,0.083380,NaN,0.000414,0.813802,0.007361,0.761715,0.001036,0.623423,0.101744
5,SD0Q4_0,0.020818,0.020946,21.065784,0.018255,1.546847,18.042187,-0.757232,0.032635,-27.525399,...,0.697720,0.084665,NaN,0.000225,0.885254,0.005220,0.710248,0.000824,0.657321,0.104943


In [27]:
import time
end_time = time.perf_counter()

timed = end_time-start_time

print(f"Experiment took {timed} seconds")
print(f"Which is {timed/60} minutes.")

Experiment took 3114.8467543 seconds
Which is 51.91411257166667 minutes.


#### TODO:
Need to fix some error with applying higher number of groups of clusters to the cluster analysis metric

In [28]:
#from PF_metrics import *
#clus = cluster_metric(pd.read_csv('../data/real/diabetes.csv'), 
#                      synthetic_data, 
#                      num_clusters=69, 
#                      metadata=settings[0]['meta']['meta_data'])

num samples data: 1536, num_klusters:69
(1536, 9)
<class 'pandas.core.frame.DataFrame'>


ValueError: Clustering algorithm could not initialize. Consider assigning the initial clusters manually.